# Job Posting Fake Detection
## Note Book 2 (_Vectorization & Modeling_)

## Installing packages

In [1]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing libraries

In [2]:
import re
from nltk.corpus import stopwords 
import string 
from nltk.stem import WordNetLemmatizer

from sklearn.ensemble import RandomForestClassifier ,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier ,ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score ,recall_score,precision_score,f1_score ,classification_report, r2_score ,confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords


      Unzipping corpora/stopwords.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

## Class For the function we might use

In [3]:
class TextPreprocessing() :
    def __init__(self, df : pd.DataFrame = pd.DataFrame) :
        self.df = df
        
        
    def Clean(self , df) :
        self.df = df
        df_copy = df.copy(deep = True)
        text_cols = list(df_copy.select_dtypes(include = ['object']).columns)
        for col in text_cols :
            for idx, text in enumerate(df_copy[col]) :
                te = []
                word = re.sub(r'(@|#)\w+' , '' , text)
                word = re.sub("[,.]", "", word)
                word = re.sub(r'https?://\S+' , '' , word)
                word = re.sub(r'(\?|!)+' , '' , word)
                word = re.sub(r"\(|\)", "", word)
                word = re.sub(r'(^\s+)' , '' , word)
                word = re.sub(r'(\s+$)' , '' , word)
                word = re.sub(r'\d+' , '' , word)
                word = word.split()
                for i in word :
                    if (i not in s_words) & (i not in punc) :
                        i = i.lower()
                        i = lmt.lemmatize(i , 'v')
                        te.append(i)
                df_copy.at[idx , col] = te
        return df_copy
    
    def Vactorize (self, df , target_name) :
        self.df = df
        self.target_name = target_name
        df_cleaned = df.copy(deep = True)
        text_cols = list(df_cleaned.select_dtypes(include = ['object']).columns)
        pos_word = {}
        neg_word = {}

        pos_df = df_cleaned[df_cleaned[target_name] == 1].reset_index(drop=True)
        neg_df = df_cleaned[df_cleaned[target_name] == 0].reset_index(drop=True)
        
        for col in text_cols :

            pos_word[col] = [word for sublist in pos_df[col] for word in sublist]
            neg_word[col] = [word for sublist in neg_df[col] for word in sublist]



        pos_freq = {}
        neg_freq = {}
        for key in pos_word.keys() :
            positive_dict = {}
            for word in pos_word[key] :
                positive_dict[word] = positive_dict.get(word , 0) + 1

            pos_freq[key] = positive_dict


        for key in neg_word.keys() :
            negative_dict = {}
            for word in neg_word[key] :
                negative_dict[word] = negative_dict.get(word , 0) + 1

            neg_freq[key] = negative_dict
            
        return pos_freq , neg_freq

        
        
        
    def Vactorization (self , df , target_name) :
        self.df = df
        self.target_name = target_name
        df_cleaned = df.copy(deep = True)
        text_cols = list(df_cleaned.select_dtypes(include = ['object']).columns)
        pos_freq , neg_freq = TextPreprocessing().Vactorize(df_cleaned , target_name)

        for col in text_cols :
            df_cleaned['{}_pos'.format(col)] = 0
            df_cleaned['{}_neg'.format(col)] = 0
            for idx, List in enumerate(df_cleaned[col]) :
                pos_frequent = 0
                neg_frequent = 0
                for word in List :
                    pos_frequent += pos_freq[col].get(word , 0)
                    neg_frequent += neg_freq[col].get(word , 0)



                df_cleaned.at[idx ,'{}_pos'.format(col)] = pos_frequent
                df_cleaned.at[idx ,'{}_neg'.format(col)] = neg_frequent
            df_cleaned.drop([col] , axis = 1 , inplace = True)
        return df_cleaned
    
    
    def fit_transform(self , df , target_name) :
        self.df = df
        self.target_name = target_name
        
        df_cleaned = TextPreprocessing().Clean(df)
        df_vact = TextPreprocessing().Vactorization(df_cleaned , target_name)
        
        return df_vact
    
    
    def Naive_Bayes(self , df, target_name) :
        self.df = df
        self.target_name = target_name
        df_naive = TextPreprocessing().Clean(df)
        pos_freq , neg_freq = TextPreprocessing().Vactorize(df_naive , target_name)
        text_cols = list(df_naive.select_dtypes(include = ['object']).columns)
        
        v_n_pos , v_n_neg = {} , {}

        for key in pos_freq.keys() :
            v_n_pos[key] = len(pos_freq[key])
            n = 0
            for word in pos_freq[key] :
                n += pos_freq[key].get(word , 0)
            v_n_pos[key]+=n
        for key in neg_freq.keys() :
            v_n_neg[key] = len(neg_freq[key])
            n = 0
            for word in neg_freq[key] :
                n += neg_freq[key].get(word , 0)
            v_n_neg[key]+=n
            
            
        prob_pos_dict = {}
        for key in pos_freq.keys():
            positive_dict = {}
            for word in pos_freq[key] :
                positive_dict[word] = (pos_freq[key].get(word , 0) + 1) / (v_n_pos[key])
            prob_pos_dict[key] = positive_dict




        prob_neg_dict = {}
        for key in neg_freq.keys():
            negative_dict = {}
            for word in neg_freq[key] :
                negative_dict[word] = (neg_freq[key].get(word , 0) + 1) / (v_n_neg[key])
                prob_neg_dict[key] = negative_dict
            
            
            
        for col in text_cols :
            df_naive['{}_probs'.format(col)] = 0
            for idx, List in enumerate(df_naive[col]) :
                score = 0
                for word in List :
                    try :
                        b = np.log((prob_pos_dict[col].get(word , 0)) / (prob_neg_dict[col].get(word , 0)))
                        if b == -float('inf') :
                            pass
                        else :
                            score +=b
                    except :
                        pass

                df_naive['{}_probs'.format(col)][idx] = score
            df_naive.drop([col] , axis = 1 , inplace = True)
            
            
        return {'probs_pos':prob_pos_dict ,'probs_neg':prob_neg_dict } , df_naive   

In [4]:
pre = TextPreprocessing()

## Read The Data

In [5]:
df = pd.read_csv("CCS_F.csv")

In [7]:
df.drop("Unnamed: 0" ,axis = 1 , inplace = True)

In [8]:
df.head()

,telecommuting,company logo exist?,has_questions,employment_type,experience required,education required,fake?,Job Info.
0,0,1,1,3,0,9,0,job title videographer editor photographer loc...
1,0,1,1,0,0,9,0,job title su chain manager location company gr...
2,0,1,1,0,0,9,0,job title digital project manager location com...
3,0,1,1,3,3,1,0,job title global communication manager locatio...
4,0,1,0,3,0,9,0,job title candidate location company lt depatm...


## Start vectorizing the Text Column

In [9]:
df_cleaned = pre.Vactorization(df , target_name = 'fake?')
df_cleaned

,telecommuting,company logo exist?,has_questions,employment_type,experience required,education required,fake?,Job Info._pos,Job Info._neg
0,0,1,1,3,0,9,0,161546283,3476394819
1,0,1,1,0,0,9,0,32258285,694238594
2,0,1,1,0,0,9,0,239181401,5146070593
3,0,1,1,3,3,1,0,266627479,5741695576
4,0,1,0,3,0,9,0,284418219,6127968530
...,...,...,...,...,...,...,...,...,...
12780,0,1,0,0,0,9,0,188289921,4054339154
12781,0,1,1,3,3,9,0,441178140,9495879441
12782,0,1,1,3,0,0,0,127343432,2740588446
12783,0,0,0,3,0,9,0,80359659,1727173775


## Modeling 

In [10]:
x = df_cleaned.drop(['fake?'] , axis = 1)
y = df_cleaned['fake?']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3 , random_state = 42)

In [11]:
list_of_models = [LogisticRegression() , KNeighborsClassifier() , 
                  DecisionTreeClassifier() ,ExtraTreeClassifier(), RandomForestClassifier(),ExtraTreesClassifier()]
classification_report = pd.DataFrame(columns=['Accuracy','F1_score','Precision','Recall'])

for model in list_of_models :
    model = model.fit(x_train , y_train)
    for i in range(2) :
        if i == 0 :
            to_pred = x_train
            pred = y_train
            title = 'Train'
            
        else :
            to_pred = x_test
            pred = y_test
            title = 'Test'
        y_pred = model.predict(to_pred)
        acc = round(accuracy_score(pred , y_pred)*100)
        f1 = round(f1_score(pred , y_pred)*100)
        prec = round(precision_score(pred , y_pred)*100)
        recall = round(recall_score(pred , y_pred)*100)
        d = pd.DataFrame(data=np.array([acc,f1,prec,recall]).reshape(1,4) 
                     , columns=['Accuracy' , 'F1_score' , 'Precision' , 'Recall'])  
        classification_report = pd.concat([classification_report , d])
        classification_report.rename( index= { 0 :'{} _ {} Details'.format(model , title) } ,inplace=True )
pd.options.display.max_rows = 15
classification_report

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Accuracy,F1_score,Precision,Recall
LogisticRegression() _ Train Details,95,0,0,0
LogisticRegression() _ Test Details,94,0,0,0
KNeighborsClassifier() _ Train Details,95,18,71,10
KNeighborsClassifier() _ Test Details,94,10,35,6
DecisionTreeClassifier() _ Train Details,100,100,100,100
DecisionTreeClassifier() _ Test Details,93,38,38,38
ExtraTreeClassifier() _ Train Details,100,100,100,100
ExtraTreeClassifier() _ Test Details,93,38,38,37
RandomForestClassifier() _ Train Details,100,100,100,100
RandomForestClassifier() _ Test Details,94,39,49,33


## Applying Grid Search on sum models

### Naive Bayes

In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Create a Gaussian Naive Bayes classifier
naive_bayes = GaussianNB()

# Define the hyperparameter grid to search
param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7]}

# Create a GridSearchCV object with the classifier, hyperparameter grid, and scoring metric
grid_search = GridSearchCV(naive_bayes, param_grid, scoring='accuracy', cv=5)

# Fit the GridSearchCV object on the training data
grid_search.fit(x_train, y_train)

# Get the best hyperparameters and the corresponding mean cross-validated score
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best hyperparameters: {'var_smoothing': 1e-09}
Best score: 0.9458039977391322


### Logistic Regression


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split

# Define the parameter grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'saga']
}

# Create the logistic regression model
lreg = LogisticRegression(solver='saga')

# Create the grid search object
grid_search = GridSearchCV(lreg, param_grid, cv=5, scoring='roc_auc')

# Fit the grid search object to the training data
grid_search.fit(x_train, y_train)

# Print the best parameters and score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not conver

Best Parameters:  {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score:  0.7484708225522595


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Create a KNN classifier
knn = KNeighborsClassifier()

# Define the hyperparameters to tune
param_grid = {'n_neighbors': [3, 5],
              'weights': ['uniform', 'distance'],
              'p': [1, 2, 3]}

# Perform grid search using cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and corresponding accuracy score
print("Best hyperparameters:", grid_search.best_params_)
print("Best accuracy score:", grid_search.best_score_)


Best hyperparameters: {'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Best accuracy score: 0.9424517301572927


### Decision Tree

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Define the model
dec_tree = DecisionTreeClassifier()

# Define the parameter grid
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Define the grid search
grid_search = GridSearchCV(dec_tree, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Best parameters:  {'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 5}
Best score:  0.9500500576146594


### Random Forest

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10]
}

# Create a random forest classifier
rfc = RandomForestClassifier()

# Create the grid search object
grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Best parameters:  {'max_depth': 15, 'min_samples_split': 10, 'n_estimators': 150}
Best score:  0.9551902220584516


### GRU

In [17]:
# x_train = np.expand_dims(x_train, axis=0)
# x_train.shape

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import GRU, Dense

# model = Sequential()

# # Add a GRU layer with 32 units and input shape (timesteps, features)
# model.add(GRU(units=32, input_shape=(x_train.shape[1], x_train.shape[2])))
# # Add a Dense layer with 1 unit and sigmoid activation function
# model.add(Dense(units=1, activation='sigmoid'))

# # Compile the model with binary crossentropy loss and adam optimizer
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Print the model summary
# print(model.summary())

In [ ]:
# def data_generator(X, y, batch_size):
#     num_batches = len(X) // batch_size
#     while True:
#         for i in range(num_batches):
#             batch_X = X[i*batch_size:(i+1)*batch_size]
#             batch_y = y[i*batch_size:(i+1)*batch_size]
#             yield batch_X, batch_y

In [ ]:
# batch_size = 26

# train_generator = data_generator(X_train, y_train, batch_size=batch_size)
# steps_per_epoch = len(X_train) // batch_size

# model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=1, verbose=1)

### Basic Sequential Model

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# # Define the model architecture
# model = Sequential()
# model.add(Dense(units=64, activation='relu', input_shape=(x_train.shape[1],)))
# model.add(Dense(units=32, activation='relu'))
# model.add(Dense(units=1, activation='sigmoid'))

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:

# # Train the model
# batch_size = 2
# model.fit(data_generator(x_train, y_train, batch_size), validation_data=(x_test,y_test), epochs=1)


## Applying the Best Model

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics

# Initialize the Random Forest Classifier with the specified hyperparameters
rfc = RandomForestClassifier(n_estimators=150, max_depth=15, min_samples_split=10)

# Fit the classifier to your training data
rfc.fit(x_train, y_train)

# Predict the classes of your test data
y_pred = rfc.predict(x_test)

conf_mat = confusion_matrix(y_test,y_pred)

print("accuracy of RF': {}".format(accuracy_score(y_pred,y_test)))
print(conf_mat)
print(metrics.classification_report(y_test,y_pred))

accuracy of RF': 0.9525547445255474
[[3606   15]
 [ 167   48]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      3621
           1       0.76      0.22      0.35       215

    accuracy                           0.95      3836
   macro avg       0.86      0.61      0.66      3836
weighted avg       0.94      0.95      0.94      3836



In [19]:
import pickle
pickle.dump(rfc, open('RF_model_CCS', 'wb'))


# Credits

---



**Directed By:**

Member 1: [Eslam Shouman](https://www.linkedin.com/in/eslam-shouman-0958091b4/)

Member 2: [Rawan ibrahim](https://www.linkedin.com/in/rawan-ibrahim-a01778268) 

Member 3: [Mohamed Mahmoud](https://www.linkedin.com/in/mohamed-mahmoud07) 


**Date of creation:**

- **29 April 2023**